# Modelo de classificação (S_GROUP_ID_1)

A seguir, apresentamos os testes realizados com os modelos de classificação durante esta Sprint. O objetivo desses modelos é prever, após a detecção de uma falha no carro, quais tipos de falhas são mais prováveis de ocorrer. Para isso, cada tipo de falha é abordado por um modelo binário. Nesta Sprint, concentramos nossos esforços na avaliação do desempenho do modelo para a classe "S_GROUP_ID_1", buscando identificar o modelo com a melhor performance. Com base nos resultados obtidos, planejamos aplicar os insights e aprimoramentos para os modelos das outras classes existentes.

Para conduzir os testes e comparar o desempenho dos diferentes modelos, utilizamos dois conjuntos de dados distintos. O primeiro conjunto de dados não incluiu as informações de torques fornecidas pelo parceiro, enquanto o segundo conjunto incorporou esses dados para treinar e aplicar o modelo. 

O processo de testes e as análises realizadas são detalhados a seguir, junto com as considerações e conclusões derivadas desses experimentos.


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,GRU, Dropout, Dense, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.optimizers import Adam


## Teste sem dados de torque

A seguir, detalhamos os testes realizados utilizando redes neurais recorrentes LSTM e GRU nos dataframe sem dados de torque. O funcionamento dessas técnicas foi explicado em profundidade na documentação da Sprint 2, onde elas foram aplicadas ao modelo principal. Devido ao seu bom desempenho nessa etapa anterior, decidimos iniciar os testes desta Sprint utilizando essas RNNs como base.

In [ ]:
# carregamento do dataframe
df = pd.read_csv("../../data/Merge_Falhas_Resultados.csv")
df["S_GROUP_ID_1"]

In [ ]:
# Converter a coluna para binário
df['S_GROUP_ID_1'] = (df['S_GROUP_ID_1'] > 0).astype(int)
df['S_GROUP_ID_1']

In [ ]:
# Separando as features (X) e o target (y)
X = df.drop(columns=['S_GROUP_ID_1', 'KNR'])  # 'KNR' é apenas um identificador, então deve ser removido
y = df['S_GROUP_ID_1']

In [ ]:
# Separando em dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Converte X_train e X_test para arrays NumPy.
X_train = np.array(X_train)
X_test = np.array(X_test)

# Reestrutura X_train e X_test para ter 3 dimensões.
# A nova forma do array será (n_samples, n_features, 1)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [ ]:
# Construção do modelo com LSTM
model_1 = Sequential()

model_1.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], 1)))
model_1.add(LSTM(50, activation='relu'))
model_1.add(Dense(1))

model_1.compile(optimizer='adam', loss='mse')

In [ ]:
# Treinamento do modelo
model_1.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
#Prever os dados de teste
y_pred_1 = model_1.predict(X_test)

# Converter as probabilidades em classes binárias (0 ou 1)
y_pred_classes_1 = (y_pred_1 > 0.5).astype(int)

# Calcular as principais métricas
accuracy_1 = accuracy_score(y_test, y_pred_classes_1)
precision_1= precision_score(y_test, y_pred_classes_1)
recall_1 = recall_score(y_test, y_pred_classes_1)
f1_1 = f1_score(y_test, y_pred_classes_1)

print(f"Accuracy: {accuracy_1:.4f}")
print(f"Precision: {precision_1:.4f}")
print(f"Recall: {recall_1:.4f}")
print(f"F1-Score: {f1_1:.4f}")


In [ ]:
df["S_GROUP_ID_1"].describe()

In [ ]:
# Construção do modelo com GRU
model_2 = Sequential()

model_2.add(GRU(50, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], 1)))
model_2.add(GRU(50, activation='relu'))
model_2.add(Dense(1, activation='sigmoid'))

model_2.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
# Treinamento do modelo
model_2.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Prever os dados de teste
y_pred_2 = model_2.predict(X_test)

# Converter as probabilidades em classes binárias (0 ou 1)
y_pred_classes_2 = (y_pred_2 > 0.5).astype(int)

# Calcular as principais métricas
accuracy_2 = accuracy_score(y_test, y_pred_classes_2)
precision_2 = precision_score(y_test, y_pred_classes_2)
recall_2 = recall_score(y_test, y_pred_classes_2)
f1_2 = f1_score(y_test, y_pred_classes_2)

print(f"Accuracy: {accuracy_2:.4f}")
print(f"Precision: {precision_2:.4f}")
print(f"Recall: {recall_2:.4f}")
print(f"F1-Score: {f1_2:.4f}")

Com base nos resultados obtidos, observamos que tanto os modelos LSTM quanto GRU apresentaram boa performance em termos de recall ao utilizar os dados de torque. No entanto, as demais métricas, como a acurácia, não tiveram o mesmo desempenho, o que nos leva a focar em melhorias nessas áreas. A próxima etapa envolve realizar testes com os dados sem torque e, em seguida, comparar os resultados dos modelos treinados com os dois conjuntos de dados. Isso nos permitirá decidir se devemos ou não manter os dados de torque e, a partir dessa decisão, testar novas técnicas para otimizar o modelo.

## Teste com dados de torque

A seguir, detalhamos os testes realizados utilizando redes neurais recorrentes LSTM e GRU nos dataframe com dados de torque. O funcionamento dessas técnicas foi explicado em profundidade na documentação da Sprint 2, onde elas foram aplicadas ao modelo principal. Devido ao seu bom desempenho nessa etapa anterior, decidimos iniciar os testes desta Sprint utilizando essas RNNs como base.

In [ ]:
# carregando o dataframe
df2 = pd.read_csv("../../data/df_torques_falhas.csv")
df2

In [ ]:
# Converter a coluna para binário
df2['S_GROUP_ID_1'] = (df2['S_GROUP_ID_1'] > 0).astype(int)
df2['S_GROUP_ID_1']



In [ ]:
# Excluindo as cplunas com dados NaN
df2 = df2.dropna()
df2.shape

In [ ]:
# Separando as features (X) e o target (y)
X2 = df2.drop(columns=['S_GROUP_ID_1', 'KNR'])  # 'KNR' é apenas um identificador, então deve ser removido
y2 = df2['S_GROUP_ID_1']


In [ ]:
# Separando em dados de treino e teste
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

In [ ]:
# Converte X_train e X_test para arrays NumPy, caso ainda não sejam.
X2_train = np.array(X2_train)
X2_test = np.array(X2_test)
# Reestrutura X_train e X_test para ter 3 dimensões.
# A nova forma do array será (n_samples, n_features, 1)
X2_train = X2_train.reshape((X2_train.shape[0], X2_train.shape[1], 1))
X2_test = X2_test.reshape((X2_test.shape[0], X2_test.shape[1], 1))


In [ ]:
# Construção do modelo com LSTM
model_1_2 = Sequential()

model_1_2.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(X2_train.shape[1], 1)))
model_1_2.add(LSTM(50, activation='relu'))
model_1_2.add(Dense(1))

model_1_2.compile(optimizer='adam', loss='mse')

In [ ]:
# Treinamento do modelo
model_1_2.fit(X2_train, y2_train, epochs=100, batch_size=32, validation_data=(X2_test, y2_test))

In [ ]:
#Prever os dados de teste
y_pred_1_2 = model_1_2.predict(X2_test)

# Converter as probabilidades em classes binárias (0 ou 1)
y_pred_classes_1_2 = (y_pred_1_2 > 0.5).astype(int)

# Calcular as principais métricas
accuracy = accuracy_score(y2_test, y_pred_classes_1_2)
precision = precision_score(y2_test, y_pred_classes_1_2)
recall = recall_score(y2_test, y_pred_classes_1_2)
f1 = f1_score(y2_test, y_pred_classes_1_2)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

In [ ]:
# Construção do modelo com GRU
model_2_2 = Sequential()

model_2_2.add(GRU(50, activation='relu', return_sequences=True, input_shape=(X2_train.shape[1], 1)))
model_2_2.add(GRU(50, activation='relu'))
model_2_2.add(Dense(1, activation='sigmoid'))

model_2_2.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
# Treinamento do modelo
model_2_2.fit(X2_train, y2_train, epochs=100, batch_size=32, validation_data=(X2_test, y2_test))

In [ ]:
# Prever os dados de teste
y_pred_2_2 = model_2_2.predict(X2_test)

# Converter as probabilidades em classes binárias (0 ou 1)
y_pred_classes_2_2 = (y_pred_2_2 > 0.5).astype(int)

# Calcular as principais métricas
accuracy = accuracy_score(y2_test, y_pred_classes_2_2)
precision = precision_score(y2_test, y_pred_classes_2_2)
recall = recall_score(y2_test, y_pred_classes_2_2)
f1 = f1_score(y2_test, y_pred_classes_2_2)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

Com base nos resultados obtidos, observamos que tanto os modelos LSTM quanto GRU tiveram um bom desempenho em termos de recall ao utilizar os dados de torque. No entanto, outras métricas, como a acurácia, não apresentaram o mesmo nível de performance, tanto com os dados de torque quanto com o dataframe sem esses dados. Ao comparar os dois dataframes, notamos que ambos tiveram um desempenho muito similar, o que indica que ambos podem ser utilizados nos modelos. No entanto, para os próximos testes, optamos por trabalhar com o dataframe que inclui os dados de torque, pois ele apresentou métricas mais consistentes e fornece um conjunto mais rico de features.

Para melhorar as métricas observadas, aplicaremos as seguintes técnicas:

- Diferentes testes de balanceamento 
- Teste de outros tipos de modelos.
- Revisão e aprimoramento do tratamento dos dados


## Últimos teste realizados nessa Sprint


Ainda essa Sprint, realizamos testes com o balanceamento por undersampling com clusterização, utilizando os centróides de cada cluster. Utilizamos essa tecnica a partir do código abaixo, utilizando X2_balanced e y2_balanced para treinar e aplicar os modelos GRU e LSTM. Entretanto, as métricas diminuiram drasticamente, indicando que devem ser realizadas melhorias ou na forma de balanceamento, ou no tramaento dos dados que estamos utilizando. Para averiguar, basta utilizar o código a seguir antes do treinamento e aplicação dos modelos na etapa de testes com dados de torque.

In [ ]:
# Separando as classes majoritária e minoritária com base no target y2
X2_majority = X2[y2 == 0]
X2_minority = X2[y2 == 1]

# Definindo o número de clusters como o tamanho da classe minoritária
n_clusters = len(X2_minority)

# Aplicando o K-Means à classe majoritária
kmeans = KMeans(n_clusters=n_clusters, random_state=1)
kmeans.fit(X2_majority)

# Pegando os centróides dos clusters
X2_majority_centroids = kmeans.cluster_centers_

# Combinando os centróides com a classe minoritária
X2_balanced = np.vstack((X2_majority_centroids, X2_minority))
y2_balanced = np.hstack((np.zeros(len(X2_majority_centroids)), np.ones(len(X2_minority))))


Além disso, tentamos também aplicar a seguinte rede neural convolucional, para testar a performance do modelo. Infelizmente, não obtivemos resultados tão positivos quanto os dos modelo GRU e LSTM, porém, pretendemos focar em pesquisar e aplicar novos tipos de rede neural convolucional que possam se adequar melhor ao problema e trazer melhores resultados na próxima Sprint.

In [ ]:
# Definir o modelo CNN
model_3 = Sequential()

# Primeira camada convolucional
model_3.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X2_train.shape[1], 1)))
model_3.add(MaxPooling1D(pool_size=2))
model_3.add(Dropout(0.3))

# Segunda camada convolucional
model_3.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
model_3.add(MaxPooling1D(pool_size=2))
model_3.add(Dropout(0.3))

# Flatten para converter dados 2D em 1D
model_3.add(Flatten())

# Camada totalmente conectada
model_3.add(Dense(64, activation='relu'))
model_3.add(Dropout(0.5))

# Camada de saída para classificação binária
model_3.add(Dense(1, activation='sigmoid'))

# Compilação do modelo
model_3.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Treinamento do modelo
model_3.fit(X2_train, y2_train, epochs=100, batch_size=32, validation_data=(X2_test, y2_test))

In [ ]:
# Prever os dados de teste
y_pred_3 = model_3.predict(X2_test)

# Converter as probabilidades em classes binárias (0 ou 1)
y_pred_classes_3 = (y_pred_3 > 0.5).astype(int)

# Calcular as principais métricas
accuracy = accuracy_score(y2_test, y_pred_classes_3)
precision = precision_score(y2_test, y_pred_classes_3)
recall = recall_score(y2_test, y_pred_classes_3)
f1 = f1_score(y2_test, y_pred_classes_3)

# Exibir as métricas
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")